# Homework 3 - Algorithmic Methods of Data Mining

In [2]:
import os
import time
import multiprocessing as mp
import ctypes
import nltk
from nltk.stem import PorterStemmer
!pip3 install tabulate
from tabulate import tabulate

import list_urls as l_u
import content_html as c_html
import anime_information as a_info
import vocabularize as voc

## 1. Data Collection

We start from the list of animes to include in our corpus of documents. In particular, we focus on the top animes listed in the first 400 pages. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).
The output of this step is a .txt file whose single line corresponds to an anime's url.

### 1.1. Get the list of animes
We extract from the main pages - the first 400 according to pagination - the urls og animes' web pages that will be retrieved in the next points.

In [3]:
path = os.getcwd()
# get the txt file with all the urls
#l_u.parallelize_extraction()

### 1.2. Get the html files from animes' urls previuosly collected
We download - via url from previuos point - and memorize all the html files in a directory organized as follows:
- a dir is created for each page, where each html in the main web page is collected.

In [ ]:
# get the html files
c_html.get_content()

### 1.3 Get the needed information about animes
Here we read each html files previously collected to get the requested information from the web pages and we create tsv files to memorize them. Each directory - so each page - contains a directory where all the tsv of the corresponding html files is memorized.

In [ ]:
# get the info about the animes
a_info.parallelize_parsing(path)

## 2. Search Engine

### 2.1 Conjunctive query

#### 2.1.1 Create indeces
After collecting the necessary data, we started focusing on the searh engine.
We initialized some needed objects and files - JSON files.

In [4]:
 # initialize object that will be needed: shared "managed" dictionaries
manager = mp.Manager()
vocabulary = manager.dict()
inverted_index = manager.dict()
complex_index = manager.dict()
docs_short = manager.dict()
#  shared "managed" counter and lock to control tasks incrementing the counter
v = manager.Value(ctypes.c_ulonglong, 0)
lock = manager.Lock()
# stemming utilities
nltk.download('punkt')
porter = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


At this point, we computed some JSON files that we will use in the next points:

    - Vocabulary: a txt file whose lines contain a pair, a pre-processed word and its unique code identifier;
    
    - Inverted_index : a JSON file whose line contain a pair, a word ID and a list of documents in which the word is present. The documents are represented as a string composed by the string document and their identification number;
    
    - Tf_complex_index : similar to inverted_index, but the list of documents contains tuples (document, tf index). At this point we only have the tf - we will get the Idf after.

In [ ]:
# get JSON files for further calculations
voc.parallelize_process_anime(path, vocabulary, inverted_index, complex_index, porter, v, lock)
voc.write_index(vocabulary, "vocabulary")
voc.write_index(inverted_index, "inverted_index")
voc.write_index(complex_index, "tf_complex_index")

Now we compute two other documents which are needed to get the cosine-similarity: 

    - TfIdf_complex index : as the tf_complex_index, but with the tfIdf index instead of the tf one. Having two step was necessary since in order to compute the Idf index we needed some overall information, which was previously missing. After we got the information (stored in the inverted_index.json), the rest could be done;
    
    - Docs_short : a complementary json file needed to compute the custom measure.

In [ ]:
# at this point you have an incomplete complex_index since the number associated to each doc is only the tf part
# get complete tfIdf complex_index
voc.get_complex_index(complex_index)
voc.write_index(complex_index, "tfIdf_complex_index")
# get index with documents as key to retrieve information on members/popularity of each anime
voc.parallelize_docs_short(path, docs_short, porter)
voc.write_docs_short(docs_short)

#### 2.1.2 Execute the query

In [5]:
# run conjunctive query and print the result as table
docs = voc.conjunctive_query(porter)
voc.print_tables(docs, path, "")

Search keys - conjunctive query : saiyan race


,animeTitle,animeDescription,Url
0,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...
1,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n
2,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
3,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...


### 2.2 Cosine-similarity ranking

In [6]:
# run cosine similarity query and print the result as table
top = voc.cosine_similarity_rank(porter, 5)
voc.print_tables(top, path, "tfIdf")

Search keys - conjunctive query : saiyan race
[(0.04113, 'document_1035'), (0.05138, 'document_364'), (0.06115, 'document_400'), (0.21109, 'document_1467')]


,animeTitle,animeDescription,Url,Similarity
0,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,0.21109
1,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...,0.06115
2,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n,0.05138
3,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...,0.04113


## 3. Custom ranking 

In [7]:
# run custom query and print the result as table
top = voc.custom_rank(porter, 5)
voc.print_tables(top, path, "tfIdf")

Search keys - conjunctive query : saiyan race
[(0.04113, 'document_1035'), (0.05138, 'document_364'), (0.06115, 'document_400'), (0.21109, 'document_1467')]


,animeTitle,animeDescription,Url,Similarity
0,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n,0.35630
1,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,0.33915
2,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...,0.29439
3,Dragon Ball Kai,"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...,0.28848


## 5. Algorithmic question
You consult for a personal trainer who has a *back-to-back sequence* of requests for appointment. A sequence of requests is of the form > 30, 40, 25, 50, 30, 20 where each number is the time that the person who makes the appointment wants to spend. You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, `[30, 50, 20]` is an acceptable solution (of duration *100*), but `[30, 40, 50, 20]` is not because *30* and *40* are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total lenght of the accepted appointments. For example, in the previous instance, the optimal solution is `[40, 50, 20]`, of total duration *110*.
1. Write an algorithm that computes the acceptable solution with the longest possible duration.
2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

#### 1. Write an algorithm that computes the acceptable solution with the longest possible duration. 


        Input:
            A: array of length n

        function alg(A):
            S <- array of length n to memorize all the sums
            dict = {}
            for i in n:
                S[i] = max(S[i-1], S[i-2] + A[i])
                if max(S[i-1], S[i-2] + A[i]) == (S[i-2] + A[i]):
                    dict[S[i-2], A[i]] = S[i]
            list <- list containing the number of the optimal solution
            individual <- 2nd index of the keys
            cumulative <- 1st index of the keys
            
            Extract all the individual values and 
            list.append(individual)

            return A, list

#### 2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

In [ ]:
def my_schedule(array):
    # Create an array we can use to memorize sums
    S = [0 for j in range(len(array))]
    
    # Create a dictionary to store all the accepted appointments
    accepted = dict()
    
    for i in range(len(array)):
        # Memorize the maximum non consecutive sum between elements in the array
        # It chooses between the last maximum sum and the one updated 
        # with the i-th element.
        S[i] = max(S[i-1], S[i-2] + array[i])
        
        # Add the accepted appointment
        if max(S[i-1], S[i-2] + array[i]) == (S[i-2] + array[i]):
            accepted[S[i-2], array[i]] = S[i]
            
    # Retrieving the numbers that produce the optimal solution
    
    # List that contains the number that produce the optimal solution
    list_of_success = []
    
    # Defining the max value of the dictionary to start the while loop
    max_value = max(accepted.values())
    
    # Defining a stop clause which is the cumulative sum or the 1st index of the keys
    stop_clause = list(accepted.keys())[list(accepted.values()).index(max_value)][0]
    
    # Stop until the cumulative value is equal to zero, which means we finished the list
    while stop_clause != 0:
        
        # Suppose we have an entry of a dictionary that is (130, 110): 240
        # Define the first index of the key, i.e. 130, as the cumulative number
        # Define the second index of the key, i.e. 110, as the individual number
        
        individual_value = list(accepted.keys())[list(accepted.values()).index(max_value)][1]
        cumulative_value = list(accepted.keys())[list(accepted.values()).index(max_value)][0]
        
        # We append the individual value to the list, since this number is part of the optimal solution
        list_of_success.append(individual_value)
        
        # Redefine the max value as
        max_value = cumulative_value
        stop_clause = cumulative_value
        
    # Reversing the list that contains the numbers of the optimal solution,
    # so they appear in the original order.
    list_of_success.reverse()
    
    return S[-1], list_of_success

In [ ]:
appointments = [30, 40, 25, 50, 30, 20]
tot, opt_sol = my_schedule(appointments)
print('The optimal solution of accepted appointments is:', opt_sol)
print('The total duration of appointments is:', tot)

In [ ]:
# Try with different lists of appointments
app1 = [30, 40, 55, 90, 90, 110]
tot, opt_sol = my_schedule(app1)
print('The optimal solution of accepted appointments is:', opt_sol)
print('The total duration of appointments is:', tot)

In [ ]:
app2 = [30, 40, 55, 90, 90, 40]
tot, opt_sol = my_schedule(app2)
print('The optimal solution of accepted appointments is:', opt_sol)
print('The total duration of appointments is:', tot)